In [11]:
import pandas as pd
import numpy as np

In [12]:
! pip install kaggle

In [13]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [14]:
! cp kaggle.json ~/.kaggle/

In [15]:
! chmod 600 ~/.kaggle/kaggle.json

In [16]:
! kaggle competitions download -c h-and-m-personalized-fashion-recommendations

100% 28.7G/28.7G [05:33<00:00, 116MB/s]
100% 28.7G/28.7G [05:33<00:00, 92.3MB/s]


In [17]:
! kaggle competitions download h-and-m-personalized-fashion-recommendations -f articles.csv 
! kaggle competitions download h-and-m-personalized-fashion-recommendations -f customers.csv
! kaggle competitions download h-and-m-personalized-fashion-recommendations -f sample_submission.csv
! kaggle competitions download h-and-m-personalized-fashion-recommendations -f transactions_train.csv


100% 4.26M/4.26M [00:00<00:00, 18.4MB/s]
100% 4.26M/4.26M [00:00<00:00, 18.4MB/s]
 91% 89.0M/97.9M [00:01<00:00, 45.8MB/s]
100% 97.9M/97.9M [00:01<00:00, 54.0MB/s]
 81% 41.0M/50.3M [00:00<00:00, 55.9MB/s]
100% 50.3M/50.3M [00:00<00:00, 56.5MB/s]
 99% 577M/584M [00:09<00:00, 83.9MB/s]
100% 584M/584M [00:09<00:00, 62.2MB/s]


In [18]:
! unzip customers.csv.zip
! unzip articles.csv 
! unzip sample_submission.csv
! unzip transactions_train.csv

Archive:  customers.csv.zip
  inflating: customers.csv           
Archive:  articles.csv.zip
  inflating: articles.csv            
Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   
Archive:  transactions_train.csv.zip
  inflating: transactions_train.csv  


In [20]:
import cv2
import numpy as np # linear algebra
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from os import listdir
from os.path import isfile, join

from termcolor import colored
from IPython.display import HTML
from PIL import Image

import warnings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('float_format', '{:f}'.format)
warnings.filterwarnings('ignore')

In [21]:
articles = pd.read_csv("articles.csv")
customer = pd.read_csv("customers.csv")
transactions = pd.read_csv("transactions_train.csv")
article_id = articles['article_id']
customer_id = customer['customer_id']
# user_item_matrix = pd.DataFrame(customer_id, columns=article_id)
# article_id = articles['article_id']
# customer_id = customer['customer_id']
# matrix = pd.DataFrame(customer_id, )


In [48]:
articles_sub = articles[['article_id','prod_name','product_type_name','product_group_name','graphical_appearance_name','colour_group_name'
                         ,'perceived_colour_value_name','perceived_colour_master_name','department_name','index_name','index_group_name'
                         ,'section_name','garment_group_name','detail_desc']]

In [49]:
#Removing spaces in all columns
for i in articles_sub.columns[1:]:
    articles_sub[i] = articles_sub[i].str.replace(" ","")

In [50]:
#Combine all information into a single column separated by space

article_cols = ['prod_name', 'product_type_name', 'product_group_name',
       'graphical_appearance_name', 'colour_group_name',
       'perceived_colour_value_name', 'perceived_colour_master_name',
       'department_name', 'index_name', 'index_group_name', 'section_name',
       'garment_group_name', 'detail_desc']
articles_sub['combined_info'] = articles_sub[article_cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [51]:
#Take out id and combined info into a new pd
articles_final = articles_sub[['article_id','combined_info']]

In [52]:
#Only 1000 products are taken because of computational issues
articles_final = articles_final.loc[:5000]

#Replace NaN with an empty string
articles_final['combined_info'] = articles_final['combined_info'].fillna('')

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')

tfidf_matrix = tfidf.fit_transform(articles_final['combined_info'])

tfidf_matrix.shape

(5001, 4732)

In [54]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim_products = linear_kernel(tfidf_matrix, tfidf_matrix)

In [55]:
indices = pd.Series(articles_final.index, index=articles_final['article_id']).drop_duplicates()

In [56]:
#Function which calculates recommendations based on the product similarity
def get_recommendations_product(title, cosine_sim=cosine_sim_products):
    # Get the index of the article that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all articles
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the articles based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar articles
    sim_scores = sim_scores[:12]

    # Get the article indices
    article_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar articles
    return articles_final['article_id'].iloc[article_indices]

In [57]:
recom = list(get_recommendations_product(108775044))
recom

[108775044,
 108775051,
 108775015,
 218354021,
 355307042,
 316441005,
 456163031,
 237347045,
 237347069,
 372008010,
 129085026,
 479227017]

In [66]:
customer['age'] = customer['age'].astype(str)
customers_sub = customer[['customer_id', 'age', 'postal_code', 'fashion_news_frequency']]
for i in customers_sub.columns[1:]:
    customers_sub[i] = customers_sub[i].str.replace(" ","")

customer_cols = ['customer_id', 'age', 'postal_code', 'fashion_news_frequency']
customers_sub['combined_info'] = customers_sub[customer_cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

customers_final = customers_sub[['customer_id','combined_info']]

customers_final = customers_final.loc[:5000]
customers_final['combined_info'] = customers_final['combined_info'].fillna('')

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(customers_final['combined_info'])
tfidf_matrix.shape

from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim_users = linear_kernel(tfidf_matrix, tfidf_matrix)

indices = pd.Series(customers_final.index, index=customers_final['customer_id']).drop_duplicates()


In [67]:
def get_similar_user(title, cosine_sim=cosine_sim_users):
    # Get the index of the article that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all articles
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the articles based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar articles
    sim_scores = sim_scores[:12]

    # Get the article indices
    customer_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar articles
    return customers_final['customer_id'].iloc[customer_indices]

In [68]:
recom = list(get_similar_user("00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657"))
recom

['00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657',
 '001b9ee636090fa2beb0965d7d9b646f221c64369786175c5763ad1a36548c88',
 '007accae76def815ed0b6f388a5b340c41722486527ff434f25dcc0f9616d47c',
 '00bd95e010cdca5eeb53adb42afe0256d3a766ff8eb41d8785af048bee83e899',
 '0014fc093a3c27277cdba16ba91bfbd37364834f48feb9c89de17a6e565767f5',
 '003540c0b3641aa2976d4a678bac602ea1ae727c530a42a362b16b11a95fa9c0',
 '0035ef3f6b5e6f0fc5c26d75afe11c364b9a319ba329559878847e860e3d48e1',
 '004a9bd306f296bf6f18014db6237a57bb1bf5bce05b0e0366a653ea1cafc40a',
 '0057aa0ad7aa5a8953cdbadb58917e5d4f0a6b53374918151f4f3d13a401380c',
 '0071a0a6690dc08042228803de04748184f9cc11985c877aa66f70fdab36b33c',
 '00b2f47f03bf90ae936c68fbcd4e8e9d58c11d69218b0e6920ba3c672d48396b',
 '000c0995dc7f3fbec41cf1cb2c535f6e2ae99c0fe351e4a40ccd037fe6dced46']

In [ ]:
print(customer_id.shape, article_id.shape)
top_customers_id = customer_id[0:10000]
top_articles_id = article_id[0:10000]
# print(top_customers_id.shape)
no_of_rows = top_customers_id.shape[0]
no_of_columns = top_articles_id.shape[0]
user_item_matrix = np.zeros((no_of_rows, no_of_columns))
df = pd.DataFrame(user_item_matrix, columns = top_customers_id, index = top_articles_id)

(1371980,) (105542,)


In [ ]:
specific_transactions_customer = transactions.loc[transactions['customer_id'].isin(top_customers_id)]
specific_transactions_overall = specific_transactions_customer.loc[specific_transactions_customer['article_id'].isin(top_articles_id)]
top_specific = specific_transactions_overall[0:100]
for index, each_transaction in top_specific.iterrows():
    c_id = each_transaction['customer_id']
    a_id = each_transaction['article_id']
    print(a_id, c_id)
    df.loc[a_id, c_id] = 1


505221004 00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2
505221001 00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2
501323011 00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4c73235dccbbc132280
531310002 0008968c0d451dbc5a9968da03196fe20051965edde7413775c4eb3be9abe9c2
529841001 0008968c0d451dbc5a9968da03196fe20051965edde7413775c4eb3be9abe9c2
501820043 000aa7f0dc06cd7174389e76c9e132a67860c5f65f970699daccc14425ac31a8
501820043 000aa7f0dc06cd7174389e76c9e132a67860c5f65f970699daccc14425ac31a8
377277001 000aa7f0dc06cd7174389e76c9e132a67860c5f65f970699daccc14425ac31a8
377277001 000aa7f0dc06cd7174389e76c9e132a67860c5f65f970699daccc14425ac31a8
397068015 001127bffdda108579e6cb16080440e89bf1250a776c6e55f56e35e9ee029a8d
464277014 002b3c0a44a22c45a8d62ea9d2b88d1a89e335f8b8400335a85c3b5845794cb9
464277014 002b3c0a44a22c45a8d62ea9d2b88d1a89e335f8b8400335a85c3b5845794cb9
427114015 00401a367c5ac085cb9d4b77c56f3edcabf25153615db982fcc7991d182e10a9
507909001 00402f4463c8dc1

In [ ]:
df.loc[a_id, c_id]

1.0

In [ ]:
from sklearn.decomposition import TruncatedSVD

# initial hyperparameter
epsilon = 1e-9
n_latent_factors = 10

# generate item lantent features
item_svd = TruncatedSVD(n_components = n_latent_factors)
item_features = item_svd.fit_transform(df.transpose()) + epsilon

# generate user latent features
user_svd = TruncatedSVD(n_components = n_latent_factors)
user_features = user_svd.fit_transform(df) + epsilon

In [ ]:
item_svd.test()